# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](https://drive.google.com/a/bu.edu/file/d/0B4bdB5WPHGsqT0lvTTZpbEdFS3c/view?usp=sharing). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

In [1]:
from collections import defaultdict

def readJson(f):
    for l in open(f):
        yield eval(l)

In [3]:
### Rating Prediction

allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)
userAndItems = defaultdict(list)
for l in readJson('amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    allRatings.append(l['overall'])
    userRatings[user].append(l['overall'])
    itemRatings[item].append(l['overall'])
#     userAndItems[user].append(item)
globalAverage = sum(allRatings) / len(allRatings)
# print (globalAverage)


In [95]:
# userAverage = {}
# for u in userRatings:
#     userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
# itemAverages = defaultdict(list)
# for r in itemRatings:
#     if (len(itemRatings[r])>0):
#         itemAverages[r].append(sum(itemRatings[r])/(len(itemRatings[r])))
#         print (sum(itemRatings[r])/(len(itemRatings[r])))
#     else:
#         itemAverages[item].append(globalAverage)

# print (itemAverages['b1e73c6d7bb534531a6e8662fe401577da0bcc2d4d7ec44acde02bc5b6702ef3'][0])
lam = 6
predictions = open("myPredictions_Rating.txt", 'w')
for l in open("amazon_reviews_Electronics/pairs_Rating.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    
    total=0
    for x in (userRatings[u]):
#         print (x)
        total+= (x -globalAverage)
    bU = (total/((len(userRatings[u])) + lam))

    total = 0    
    for x in itemRatings[i]:
        total += (x- globalAverage - bU)
    bI = (total/((len(itemRatings[i])) + lam))
    

    result = globalAverage + bU + bI 
    predictions.write(u + '-' + i + ',' + str(result) + '\n')

predictions.close()


In [21]:
itemCategories = defaultdict(list)
for l in readJson('amazon_reviews_Electronics/meta.json'):
    item, categories = l['asin'],l['categories']
    itemList = (categories.strip('[]').split(','))
    itemList = [x.strip('\' ') for x in itemList]
    itemCategories[item].append(itemList)


In [22]:
setCat = set()
for i in itemCategories:
    for j in itemCategories[i]:
        for x in j:
            x = (x.strip(']')).strip('\'')
            setCat.add(x.strip('[]'))


In [24]:
# import pandas as pd
# import numpy as np
# z = np.zeros(((len(allUsers)),(len(setCat))))
# #create a data frame of the items and what categories its in
# df = pd.DataFrame(data=z,index=(list(allUsers)),columns=(list(setCat)))


In [29]:
# ### Purchasing baseline: just rank which items are popular and which are not, and return '1' if an item is among the top-ranked

# itemCount = defaultdict(int)
# totalPurchases = 0
# for l in readJson('amazon_reviews_Electronics/train.json'):
#     user,item = l['reviewerID'],l['asin']
    
#     itemCount[item] += 1
#     totalPurchases += 1

# mostPopular = [(itemCount[x], x) for x in itemCount]
# mostPopular.sort(reverse=True)

# return1 = set()
# count = 0
# for ic, i in mostPopular:
#     count += ic
#     return1.add(i)
#     if count > totalPurchases/2: break



In [30]:
# make two matrices 

# the users and items they bought
# items and who bought it
# 1 with items and the categories
userItemsPurchased = defaultdict (list)
itemsPurchasedBy = defaultdict(list)
allUsers = set()
for l in readJson('amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    allUsers.add(user)
    userItemsPurchased[user].append(item)
    itemsPurchasedBy[item].append(user)


In [98]:
# Code that works for task 2 

predictions = open("myPredictions_Purchase.txt", 'w')
for l in open("amazon_reviews_Electronics/pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    
    listOfBuyers = (itemsPurchasedBy[i])
    if (len(listOfBuyers) > 1):
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")
predictions.close()


In [90]:

### Helpfulness baseline: similar to the above. Compute the global average helpfulness rate, and the average helpfulness rate for each user

# how many people voted that the review was helpful
# why will a review be considered helpful?
allHelpful = []
userHelpful = defaultdict(list)
itemScore = defaultdict(list)
itemHelpful = defaultdict(list)

for l in readJson('amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    allHelpful.append(l['helpful'])
    userHelpful[user].append(l['helpful'])
    itemScore[item].append(l['overall'])
    itemHelpful[item].append(l['helpful'])
    
averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])
userRate = {}
for u in userHelpful:
    userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / sum([x['outOf'] for x in userHelpful[u]])
itemRate = {}
for h in itemHelpful:
    itemRate[h] = sum([x['nHelpful'] for x in itemHelpful[h]]) * 1.0 / sum([x['outOf'] for x in itemHelpful[h]])


In [84]:

# # calculate the bias
# predictions = open("myPredictions_Helpful.txt", 'w')
# for l in open("amazon_reviews_Electronics/pairs_Helpful.txt"):
#     if l.startswith("reviewerID"):
#     #header
#         predictions.write(l)
#         continue
#     u,i,outOf = l.strip().split('-')
# #     print (avgScore)
# #     break

#     outOf = int(outOf)
#     if (u in userRate) and (i in itemRate):
# #         print ((userRate[u]+ avgScore)/2)
# #         break
        
#         predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*(((averageRate+userRate[u])+itemRate[i])/3)) + '\n')
#     else:
#         predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*averageRate) + '\n')
# predictions.close()


In [115]:

userRatedItem = defaultdict(list)
itemRated = defaultdict(list)
for line in readJson('amazon_reviews_Electronics/helpful.json'):
    user, item, rating = line['reviewerID'], line['asin'], line['overall']
    userRatedItem[user].append(item)
    itemRated[item].append([user,rating])
    
predictions = open("myPredictions_Helpful.txt", 'w')
for l in open("amazon_reviews_Electronics/pairs_Helpful.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')

    if u in userRatedItem and i in itemRated:
        for user,rating in itemRated[i]:
            if u == user:
                userRating = rating
                break
            else:
                userRating=-1
            

    outOf = int(outOf)
    if userRating == 1 or userRating == 5:
            predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str((outOf*1)) + '\n')
    elif userRating == 4 or userRating == 2:
            predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str((outOf*.95)) + '\n')
    else:
        if u in userRate and i in itemRate:
            predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*(((averageRate+userRate[u])+itemRate[i])/3)) + '\n')
        else:
            predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*averageRate) + '\n')

#         predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*averageRate) + '\n')
predictions.close()


## Files

# **train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predict whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs505-amazon-rating-prediction) click here to [join](https://kaggle.com/join/rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs505-amazon-purchase-prediction) click here to [join](https://kaggle.com/join/purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs505-amazon-helpful-prediction) click here to [join](https://kaggle.com/join/helpful)

## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**10pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

-----------------

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015